In [1]:
!pip install folium  #installing folium

In [2]:
#libraries 
import numpy as np
import pandas as pd
import folium
import requests

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as  colors
from pandas.io.json import json_normalize

In [3]:
#coordinates of shimla 
latitude = "31.1048"
longitude = "77.1734"
print("The coordinates of Shimla are {}, {}.".format(latitude,longitude))

The coordinates of Shimla are 31.1048, 77.1734.


In [4]:
#displaying shimla on a map
shimla_map =folium.Map(location = [latitude,longitude], zoom_start =14.70)
shimla_map

In [5]:
#foursquare API and stuff
cli_id = "PZAUUPBXMCOPLFIV15VL4BTTADK3XZ0CZGIX0T0AXQ1PAGYO"
cli_sec = "L3DG2NCFUCIZLYRLC24Z2UUD3PTJLDTHMXG0YYRH0Q1U2X0I"
rad = 10000  #that's 5km
no_of_venues = 50
ver = "20191106"  #current date

In [6]:
#defining the category function straight from coursebook implementation
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) ==0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
pd.set_option("display.max_rows",None)

offset= 0
total_venues =0
fs_venues = pd.DataFrame(columns = ['name','categories','lat','long'])


In [8]:
while(True):
    url = ("https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&offset={}").format(cli_id,cli_sec,ver,latitude,longitude,rad,no_of_venues,offset)
    result =requests.get(url).json()
    venues_fetched = len(result['response']['groups'][0]['items'])
    total_venues = total_venues + venues_fetched
    print("Total {} venues fetched within a total radiues of {} km".format(venues_fetched,rad/1000))
    venues = result['response']['groups'][0]['items']
    venues = json_normalize(venues)
    
    #filter the columns
    filtered_columns =['venue.name','venue.categories','venue.location.lat','venue.location.lng']
    venues =  venues.loc[:,filtered_columns]
    
    #filter the category for each row
    venues['venue.categories'] = venues.apply(get_category_type, axis =1)
    
    #clean the column names
    venues.columns = [col.split(".")[-1] for col in venues.columns]
    fs_venues = pd.concat([fs_venues,venues], axis =0, sort= False)
    if(venues_fetched <50):
        break
    else:
        offset = offset +50
    fs_venues = fs_venues.reset_index(drop =True)
    print("\nTotal {} venues fetched".format(total_venues))

Total 27 venues fetched within a total radiues of 10.0 km


In [9]:
#use zomato API

In [ ]:
headers = {'user-key':'0ce13f014dafcc1baaea30e15b4055c5'}
venues_information = []

for index, row in fs_venues.iterrows():
    print("Fetching data for venue: {}".format(index+1))
    venue = []
    url = ('https://developers.zomato.com/api/v2.1/search?q={}' + 
          '&start=0&count=1&lat={}&lon={}&sort=real_distance').format(row['name'], row['lat'], row['lng'])
    result = requests.get(url,headers = headers).json()
    if(len(result['restaurants']) > 0):
        venue.append(result['restaurants'][0]['restaurant']['name'])
        venue.append(result['restaurants'][0]['restaurant']['location']['latitude'])
        venue.append(result['restaurants'][0]['restaurant']['location']['longitude'])
        venue.append(result['restaurants'][0]['restaurant']['average_cost_for_two'])
        venue.append(result['restaurants'][0]['restaurant']['price_range'])
        venue.append(result['restaurants'][0]['restaurant']['user_rating']['aggregate_rating'])
        venue.append(result['restaurants'][0]['restaurant']['location']['address'])
        venues_information.append(venue)
    else:
        venues_information.append(np.zeros(6))
    
zomato_venues = pd.DataFrame(venues_information, 
                                  columns = ['venue', 'latitude', 
                                             'longitude', 'price_for_two', 
                                             'price_range', 'rating', 'address'])

Fetching data for venue: 1
Fetching data for venue: 2
Fetching data for venue: 3
Fetching data for venue: 4
Fetching data for venue: 5
Fetching data for venue: 6
Fetching data for venue: 7
Fetching data for venue: 8
Fetching data for venue: 9
Fetching data for venue: 10
Fetching data for venue: 11
Fetching data for venue: 12
Fetching data for venue: 13
Fetching data for venue: 14
Fetching data for venue: 15
Fetching data for venue: 16
Fetching data for venue: 17
Fetching data for venue: 18
Fetching data for venue: 19
Fetching data for venue: 20
Fetching data for venue: 21
Fetching data for venue: 22
Fetching data for venue: 23
Fetching data for venue: 24
Fetching data for venue: 25
Fetching data for venue: 26
Fetching data for venue: 27


In [ ]:
#Data Cleaning

In [1]:
chandigarh_map = folium.map(location = [latitude,longitude],zoom_start =14.70) 
for name,latitude,longitude in zip(fs_venues['name'],fs_venues['lat'],fs_venues['lng']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [latitude, longitude],
    radius = 5,
    popup = label,
    color = 'red',
    fill=True,
    fill_color = '#FF0000',
    fiill_opacity =0.5,
    parse_html =False).add_to(chandigarh_map)
    
chandigarh_map

NameError: name 'folium' is not defined